In [1]:
## inspired by https://commercedataservice.github.io/tutorial_biz_dynamics/
from IPython.display import display
import io, requests, zipfile
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py

from plotly import __version__
## print (__version__) ## requires version >= 1.9.0

## Generating Offline Graphs within Jupyter Notebook
## https://plot.ly/python/offline/
init_notebook_mode(connected=True)


In [2]:
def get_data(msa_code, var_dict, para_dict):
    ## catenate keys using str.join() method.
    r = requests.get('http://api.census.gov/data/timeseries/bds/firms?get=' \
                     + ','.join(var_dict.keys()) \
                     + '&for=metropolitan+statistical+area:'\
                     + str(msa_code), \
                    params=para_dict)
    
    ## print(r)
    ## when url returns empty content, return None
    if r.content is b'': 
        return None
    else:
        ## read in data
        data = pd.DataFrame(r.json())
        
        ## get columns names from first row and map them to the given names in var_dict
        columns = data.iloc[0,:-len(para_dict)].map(var_dict)
        
        ## data we are interested in 
        data = data.iloc[1:,:-len(para_dict)]
        
        ## rename dataframe columns
        data.columns = columns
        
        return data

In [3]:
## request data via https
## https://www2.census.gov/econ/susb/data/
r = requests.get('https://www2.census.gov/econ/susb/data/msa_codes_2007_to_2011.txt').content

## read and parse data
msa_code = pd.read_table(io.StringIO(r.decode('utf-8')), header=3, sep='   ') 

## rename columns
msa_code.columns = ['code','name'] 

## get rid of micropolitan statistical areas
msa_code = msa_code[msa_code['name'].str.contains('Metropolitan', case = False)]

## clean up names
msa_code['name'] = msa_code['name'].str.replace(' Metropolitan Statistical Area', '') 

## function to clean up MSA names, only keep the fist city and fist state
def name_clean(s):
    s_list = s.split(', ')
    cities = s_list[0]
    states = s_list[-1]
    return ' '.join([cities.split('-')[0], states.split('-')[0]])

## map the name_clean function to all MSA
msa_code['name'] = msa_code['name'].map(name_clean) 

/Users/udothemath1984/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [4]:
api_key = '377a44f6cb37b1197c1b43db457bbb4e1330db9f'

In [5]:
def survival_rate(name, start_year):
    
    #fage4 is the firm age in a given year. Letters a through f represent years 0 through 5
    fage4_values = ['a', 'b', 'c', 'd', 'e', 'f']
    
    #The data only covers up to 2013 (as of now), so we will limit the fage4_values to ones within 2013 minus start_year
    if 2013 - start_year < 6:
        fage4_values = fage4_values[0:(2013-start_year + 1)]

    #var_dict contains the variables and their labels
    var_dict = {'estabs': 'Establishments',
                'emp': 'Employment',
                'firms': 'Firms'}
    
    #set up empty array to accept results from get_data
    result = []
    
    #grab the msa code based on the 'name' provided in the input parameters
    code = msa_code[msa_code['name'].str.contains(name, case = False)]['code'].values[0]
    ## print(code)  # 35620 for survival_rate('New York', 2009)
    ## print(fage4_values)
    #Loop from start year to the 5th year (or the cap year if end years exceed 2013)
    for i in range(len(fage4_values)):
        para_dict = {'fage4': fage4_values[i], 'time': start_year+i }
        result.append(get_data(code, var_dict, para_dict))
        ## print(para_dict)
        
    #The code was returning an error as not all variables were integer friendly (e.g. there was a phantom column of letters)
    #Added in a drop statement to keep only variables 0:4 
    df = pd.concat(result).iloc[:, 0:3].astype(int)
    df.index = range(start_year,start_year+len(fage4_values))

    ## print(df.index)
    #Calculate point survival rate
    #Step 1: Create denominator dataframe
    #Shift rows up 1
    df2 = df.shift(1)
    
    #Replace blank row with 
    df2.iloc[[0]] = df.iloc[[0]]
    
    #Step 2: Divide numerator (df) by denominator (df2)
    df3 = df/df2

    #Step 3: Calculate cumulative product on instantaneous survival rate table
    df4 = df3.cumprod()*100

    
    ### start plotting using plotly
    data = []
    for label in df4.columns:
        trace = go.Scatter(
            x = df4.index,
            y = df4[label].values,
            name = label
        )
        data.append(trace)

    layout = dict(title = 'Business Survival Rates, '+ name +' Metropolitan Area, Year: '+str(start_year),
                  yaxis = dict(title = 'survival rate (%)'),
                  xaxis = dict(title = 'year', nticks = len(df4)),
                  )

    fig = dict(data=data, layout=layout)
    iplot(fig)



In [6]:
## draw down zip file, unzip, read in txt with specified encoding
r = requests.get("http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2015_Gazetteer/2015_Gaz_cbsa_national.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
geo = pd.read_table(z.open('2015_Gaz_cbsa_national.txt'), encoding = "ISO-8859-1")


In [7]:
## clean the columns names and change to lower case
## https://www.census.gov/geo/maps-data/data/gazetteer2015.html
geo.columns = [field.rstrip().lower() for field in geo.columns]

## get rid of micropolitan statistical areas and clean the names the same as msa_code 
geo = geo[geo['name'].str.contains('Metro', case = False)]
geo['name'] = geo['name'].str.replace(' Metro Area', '')
geo['name'] = geo['name'].map(name_clean)

In [8]:
display(geo.head(5))

,csafp,geoid,name,cbsa_type,aland,awater,aland_sqmi,awater_sqmi,intptlat,intptlong
2,184.0,10420,Akron OH,1,2331619578,62018442,900.243,23.945,41.146639,-81.350110
3,440.0,10540,Albany OR,1,5932815078,49654169,2290.673,19.172,44.488898,-122.537208
4,104.0,10580,Albany NY,1,7282106737,172589034,2811.637,66.637,42.787920,-73.942348
7,106.0,10740,Albuquerque NM,1,24041256743,37921741,9282.382,14.642,35.116603,-106.456535
9,408.0,10900,Allentown PA,1,3763879943,58581593,1453.242,22.618,40.789339,-75.398158


### Population estimates

In [9]:
## http://mcdc.missouri.edu/data/popests/CBSA-EST2014-alldata.csv
## read in data with specified encoding
pop = pd.read_csv("http://mcdc.missouri.edu/data/popests/CBSA-EST2014-alldata.csv", 
                  encoding = "ISO-8859-1")
pop = pop[pop['LSAD']=='Metropolitan Statistical Area']
pop = pop[['CBSA','POPESTIMATE2014']]
pop.columns = ['geoid', 'population']

In [10]:
display(pop.head(5))

,geoid,population
3,10180.0,168592.0
7,10420.0,703825.0
10,10500.0,154925.0
16,10540.0,119356.0
18,10580.0,880167.0


In [11]:
geo = geo.merge(pop, how='inner', left_on='geoid', right_on='geoid')

#Merge msa_code to geo
msa_code = msa_code.merge(geo[['name','intptlat','intptlong','population', 'aland_sqmi']],how='left', left_on='name', right_on='name')
msa_code = msa_code.dropna()

In [12]:
display(msa_code.head(5))

,code,name,intptlat,intptlong,population,aland_sqmi
0,10180,Abilene TX,32.452022,-99.718743,168592.0,2743.479
1,10420,Akron OH,41.146639,-81.350110,703825.0,900.243
2,10500,Albany GA,31.589303,-84.174913,154925.0,1932.597
3,10580,Albany NY,42.787920,-73.942348,880167.0,2811.637
4,10740,Albuquerque NM,35.116603,-106.456535,904587.0,9282.382


In [20]:
## specify starting year of analysis
start_year = 2008

## letters indicating firm age
fage4_values = ['a', 'b', 'c', 'd', 'e', 'f'] 

## deisred variables
var_dict = {'firms': 'firms',
            'emp': 'jobs'} 

## empty dataframe as placeholder
df = pd.DataFrame(columns = ['name', 'population', 'aland_sqmi', 'lat', 'lon', '5-year firm survival', 
                             '5-year job survival', 'number of jobs', 'number of firms'])

print('Fetching data for...')
## iterate through every MSA with a population bigger than 250,000
count = 0
for idx, row in msa_code[msa_code['population']>=2.5e5].iterrows():
    
    ## code and name of current MSA
    code = row['code']
    ##print('    '+row['name'])
    
    ## place holder for results
    result = []
    
    ## iterate through age 0 - 5
    for i in range(6):
        para_dict = {'fage4': fage4_values[i], 'time': start_year + i, 'key': api_key}
        result.append(get_data(code, var_dict, para_dict))

    ## check for empty results
    if any([d is None for d in result]):
        continue
        
    #The code was returning an error as not all variables were integer friendly (e.g. there was a phantom column of letters)
    #Added in a drop statement to keep only variables 0:4 
    df0 = pd.concat(result).iloc[:, 0:3].astype(int)
    df0.index = range(start_year,start_year+len(fage4_values))

    #Calculate point survival rate
    #Step 1: Create denominator dataframe
    #Shift rows up 1
    df1 = df0.shift(1)
    
    #Replace blank row with 
    df1.iloc[[0]] = df0.iloc[[0]]
    
    #Step 2: Divide numerator (df) by denominator (df2)
    df2 = df0/df1
    

    #Step 3: Calculate cumulative product on instantaneous survival rate table, keep only year 5
    df3 = df2.cumprod()*100
    
    ## copy the initial number of jobs and firms
    df.loc[code, ['number of jobs', 'number of firms']] = df0.iloc[[0]][['jobs', 'firms']].values

     ## copy the initial survival probs
    df.loc[code, ['5-year firm survival', '5-year job survival']] = df3.iloc[[5]][[ 'firms','jobs']].values

    ## copy the namem population and location of MSA
    df.loc[code, ['name', 'population', 'aland_sqmi', 'lat', 'lon']] = row[['name', 'population', 'aland_sqmi', 'intptlat','intptlong']].values 
    
    ## Testing termination
#     count += 1
#     if count >= 3:
#         break
print('Done!')



Fetching data for...
Done!


In [14]:
# https://stackoverflow.com/questions/28754658/whats-the-fastest-way-to-pickle-a-pandas-dataframe
## build-in pickle method
df.to_pickle('df_city_info.p')

# import pickle
# df.to_pickle()

In [30]:
display(df.head(3))

,name,population,aland_sqmi,lat,lon,5-year firm survival,5-year job survival,number of jobs,number of firms,firm_population_ratio,job_population_ratio
10420,Akron OH,703825,900.243,41.1466,-81.3501,49.9401,68.2726,5708,835,1.18637,8.10997
10580,Albany NY,880167,2811.64,42.7879,-73.9423,48.2935,104.419,5069,1172,1.33157,5.75913
10740,Albuquerque NM,904587,9282.38,35.1166,-106.457,43.7547,76.0735,7126,1321,1.46033,7.87763


In [29]:
per_unit = 1000
df['firm_population_ratio'] = per_unit* df['number of firms'] / df['population']    
df['job_population_ratio'] = per_unit * df['number of jobs'] / df['population'] 

In [58]:
def map_bubblePlot(size_field, color_field): 
    scaling_for_job = 2000
    
    ## white     : rgb(255, 255, 255)
    ## redish    : rgb(255, 0, 0)
    ## yellowish : rgb(255, 255, 0)
    ## bluish    : rgb(0, 0, 255)
    ## cryan     : rgb(0, 255, 255)
    ## yellow    " rgb(255, 255, 0)

    bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.9, 0.97, 1]
    n_color_segment = len(bins) - 1
    ## print(n_color_segment)
    each_increment = float(256 / n_color_segment) 
    colors = ["rgb(255, {}, {})".format(255-i*each_increment, 255-i*each_increment) \
              for i in range(0,n_color_segment)]
    
    qs = df[color_field].quantile(bins).values
    ## print(qs)
    color_map = pd.qcut(df[color_field], bins, labels=[i for i in range(n_color_segment)])
               
    df['text'] = (df['name'] 
    + '<br>population: ' + (df['population']/1e6).map(lambda x: '%2.1f' % x) + ' million'
    + '<br>' + size_field + ': ' + df[size_field].map(lambda x: '{:,}'.format(x))
    + '<br>' + color_field + ': ' + df[color_field].map(lambda x: '%2.2f' % x) + '%')

    trace0 = go.Scatter(
        name='Area Size: '+ size_field
             + '<br>' + 'Color Darkness: ' + color_field,
        x=list(df['population'].values),
        y=list(df['aland_sqmi'].values),
        mode='markers',
        marker=dict(
            color=["rgb(255, {}, {})".format(255-i*each_increment, 255-i*each_increment) \
              for i in color_map],
    #         opacity=[1, 0.8, 0.6, 0.4],
            size=df[size_field].values/scaling_for_job
        ),
        text=df['text']
    )
    layout = dict(
    #title = size_field+' created in 2008, grouped by '+color_field,
        showlegend = True,
        xaxis=dict(
            title="Number of population",
            ),
        yaxis=dict(
            title="Size of city (mi^2)",
        ),
        margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4,
        ),

    )
    
    data = [trace0]    
    fig = dict( data=data, layout=layout)
    iplot(fig, filename='bubblechart-color')
        

In [59]:
def map_plot(size_field, color_field):
    
    ## value to scale down bubble size
    scale = df[size_field].max()/4e3
    
    
    ## https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.quantile.html
    ## setting quantiles
    bins = [0, 0.03, 0.1, 0.5, 0.9, 0.97, 1]
    
    ## setting colors
    ##colors = ['#8c510a', '#d8b365', '#f6e8c3', '#c7eae5', '#5ab4ac', '#01665e']
    n_color_segment = 6
    each_increment = float(256 / n_color_segment) 
    ## white     : rgb(255, 255, 255)
    ## redish    : rgb(255, 0, 0)
    ## yellowish : rgb(255, 255, 0)
    ## bluish    : rgb(0, 0, 255)
    ## cryan     : rgb(0, 255, 255)
    ## yellow    " rgb(255, 255, 0)
#     colors = ["rgb(255,{},{})".format(255-i*each_increment, 255-i*each_increment) \
#               for i in range(0,n_color_segment)]

    colors = ["rgb(255, {}, {})".format(255-i*each_increment, 255-i*each_increment) \
              for i in range(0,n_color_segment)]
    ##print(colors)
    ## place holder for msa traces 
    msas = []
    
    ## text to show when mouse move over
    df['text'] = (df['name'] 
        + '<br>population: ' + (df['population']/1e6).map(lambda x: '%2.1f' % x) + ' million'
        + '<br>' + size_field + ': ' + df[size_field].map(lambda x: '{:,}'.format(x))
        + '<br>' + color_field + ': ' + df[color_field].map(lambda x: '%2.2f' % x) + '%')
    
    ## calculate the corresponding values for each quantile
    qs = df[color_field].quantile(bins).values
    
    ##print(qs)
    
    ## iterate through each group
    for lower, upper, color in zip(qs[:-1], qs[1:], colors):
        
        ## handling lower bound
        if color==colors[0]: 
            df_sub = df[(df[color_field]<upper)]
            
            ## format the value for display
            name = '< {0:.0f}%'.format(upper)
            
        ## handling upper bound
        elif color==colors[-1]: 
            df_sub = df[(df[color_field]>lower)]
            name = '> {0:.0f}%'.format(lower)
        ## other groups    
        else: 
            df_sub = df[(df[color_field]>lower)&(df[color_field]<=upper)]
            name = '{0:.0f}% - {1:.0f}%'.format(lower,upper)
        
        ## put data into a dictionary in plotly format
        msa = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df_sub['lon'],
            lat = df_sub['lat'],
            text = df_sub['text'],
            marker = dict(
                size = df_sub[size_field]/scale,
                color = color,
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area'
            ),
            name = name )
        
        ## append current data to placeholder
        msas.append(msa)
    
    ## setting figure title and layout
    layout = dict(
        title = size_field + ' created in 2008, grouped by '+color_field,
        showlegend = True,
        margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
        ),
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'white',
            subunitwidth=0.5,
            countrywidth=0.5,
            subunitcolor="gray",
            countrycolor="gray",
        ),
    )

    fig = dict( data=msas, layout=layout )
    iplot(fig)

In [60]:
map_plot('number of jobs', '5-year job survival')
map_bubblePlot('number of jobs', '5-year job survival')

In [31]:
display(df.head(3))

,name,population,aland_sqmi,lat,lon,5-year firm survival,5-year job survival,number of jobs,number of firms,firm_population_ratio,job_population_ratio
10420,Akron OH,703825,900.243,41.1466,-81.3501,49.9401,68.2726,5708,835,1.18637,8.10997
10580,Albany NY,880167,2811.64,42.7879,-73.9423,48.2935,104.419,5069,1172,1.33157,5.75913
10740,Albuquerque NM,904587,9282.38,35.1166,-106.457,43.7547,76.0735,7126,1321,1.46033,7.87763


In [121]:
df_job_pop = df.sort_values(by=['job_population_ratio'], ascending=False)

df_firm_pop = df.sort_values(by=['firm_population_ratio'], ascending=False)

In [122]:
display(df_job_pop.head(3))
display(df_firm_pop.head(3))

,name,population,aland_sqmi,lat,lon,5-year firm survival,5-year job survival,number of jobs,number of firms,firm_population_ratio,job_population_ratio,text
46540,Utica NY,296615,2623.94,43.3323,-75.1732,55.3846,94.9391,5908,325,1.0957,19.9181,Utica NY<br>population: 0.3 million<br>number ...
13140,Beaumont TX,405427,3034.34,30.3478,-94.1204,51.1737,31.5114,6861,426,1.05074,16.9229,Beaumont TX<br>population: 0.4 million<br>numb...
29820,Las Vegas NV,2.06968e+06,7891.48,36.2143,-115.014,40.0355,61.8202,29293,3944,1.90561,14.1534,Las Vegas NV<br>population: 2.1 million<br>num...


,name,population,aland_sqmi,lat,lon,5-year firm survival,5-year job survival,number of jobs,number of firms,firm_population_ratio,job_population_ratio,text
14500,Boulder CO,313333,726.34,40.095,-105.398,49.3348,89.5634,3344,902,2.87873,10.6724,Boulder CO<br>population: 0.3 million<br>numbe...
48900,Wilmington NC,272548,1061.4,34.426,-77.8896,43.2051,69.6696,3541,780,2.86188,12.9922,Wilmington NC<br>population: 0.3 million<br>nu...
33100,Miami FL,5.92982e+06,5075.49,26.1018,-80.4788,42.5921,60.1716,70151,14660,2.47225,11.8302,Miami FL<br>population: 5.9 million<br>number ...


In [130]:
# https://plot.ly/python/horizontal-bar-charts/
# import plotly.plotly as py
# import plotly.graph_objs as go

top_n = 10

my_desired_property = '5-year firm survival'

## white     : rgb(255, 255, 255)
## redish    : rgb(255, 0, 0)
## yellowish : rgb(255, 255, 0)
## bluish    : rgb(0, 0, 255)
## cryan     : rgb(0, 255, 255)
## yellow    " rgb(255, 255, 0)

bins = [0, 0.03, 0.05, 0.1, 0.2, 0.25, 0.5, 0.9, 0.97, 1]
n_color_segment = len(bins) - 1
## print(n_color_segment)
each_increment = float(256 / n_color_segment) 

color_map = pd.qcut(df_firm_pop[my_desired_property], bins, labels=[i for i in range(1, n_color_segment+1)])

qs = df_firm_pop[my_desired_property].quantile(bins).values
# print(qs)
# print(df_firm_pop[my_desired_property][:top_n])
# print(color_map[:top_n])

data = [go.Bar(
            x=df_firm_pop['firm_population_ratio'][:top_n],
            y=df_firm_pop['name'][:top_n],
            orientation = 'h',
            text = ["{:.2f}".format(i) for i in df_firm_pop[my_desired_property]],
            textfont=dict(
                family='sans serif',
                size=18,
                color='#41f4be'
            ),
            textposition = 'auto',
            marker=dict(
              color=["rgb({}, {}, 255)".format(255-i*each_increment, 255-i*each_increment) \
              for i in color_map],
        ),
)]

layout = dict(
        title = "Color with text indicates the 5-year firm survival rate (Higher the better)",
        showlegend = True,
        xaxis=dict(
            title="firm / population (per 10^3)",
            ),
        yaxis=dict(
            title="",
        ),
        margin=go.Margin(
        l=150,
        r=50,
        b=100,
        t=100,
        pad=4,
        ),

    )
    
fig = dict( data=data, layout=layout)

layout['showlegend'] = False
iplot(fig, filename='horizontal-bar')

The previous plot shows the city with highest firm/population ratio among the large city (population $\ge$ 25,000). Every 1000 people, there are $\approx$ 2.88 firms in Boulder, CO. The 5-year firm survival rate at Boulder is 49.33 %, indicating its healthy survival strength. In general, the 10 top most firm/population city have high firm survival rate. Is there any other reason behind this observation?

In [133]:
# https://plot.ly/python/horizontal-bar-charts/
# import plotly.plotly as py
# import plotly.graph_objs as go

my_interest = '5-year job survival'

top_n = 10

bins = [0, 0.03, 0.05, 0.1, 0.2, 0.25, 0.5, 0.9, 0.97, 1]
n_color_segment = len(bins) - 1
## print(n_color_segment)
each_increment = float(256 / n_color_segment) 

color_map = pd.qcut(df_job_pop[my_interest], bins, labels=[i for i in range(1, n_color_segment+1)])
qs = df_job_pop[my_interest].quantile(bins).values
# print(qs)
# print(df_job_pop[my_interest][:top_n])
# print(color_map[:top_n])

df_format = df_job_pop.style.format("{:.2%}")

data = [go.Bar(
            x=df_job_pop['job_population_ratio'][:top_n],
            y=df_job_pop['name'][:top_n],
            text = ["{:.2f}".format(i) for i in df_job_pop[my_interest]],
            textfont=dict(
                family='sans serif',
                size=18,
                color='#41f4be'
            ),
            textposition = 'auto',
            orientation = 'h',
            marker=dict(
              color=["rgb({}, {}, 255)".format(255-i*each_increment, 255-i*each_increment) \
              for i in color_map],
        ),
)]

layout = dict(
        title = "Color with text indicates the 5-year job survival rate (Higher the better)", 
        showlegend = True,
        xaxis=dict(
            title="job / population (per 10^3)",
            ),
        yaxis=dict(
            title="",
        ),
        margin=go.Margin(
        l=150,
        r=50,
        b=100,
        t=100,
        pad=4,
        ),

    )
    
fig = dict( data=data, layout=layout)

layout['showlegend'] = False
iplot(fig, filename='horizontal-bar_job')

The previous plot shows the city with highest job/population ratio among the large city (population $\ge$ 25,000). Every 1000 people, there are $\approx$ 19.9 % jobs in Utica, NY. The 5-year job survival rate is 94.94, indicating the healthy employment environment. In contrast, the 5-year job survival rate is very low in Beaumont, TX, even though it has high job/population ratio (16.92 per 1000 population). The further investigation could reveal the underneath reason, which is critical to improve the job market at low job survival rate city. 